### Análisis Exploratorio: San Francisco Bay Bike Rental

### Repositorio: https://github.com/bzuker/tp-datos

### Importamos los paquetes

In [ ]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(context='notebook', font_scale=1.2)

### Importamos los datos

In [ ]:
trips = pd.read_csv('../input/trip.csv')
weather = pd.read_csv('../input/weather.csv')
stations = pd.read_csv('../input/station.csv')
#status = pd.read_csv('../input/status.csv', low_memory= False)

### Exploramos Trips

In [ ]:
trips.head()

In [ ]:
trips.dtypes
trips.describe()
trips.duration.describe()
trips.shape

In [ ]:
# Paso duration a minutos.
trips.duration /= 60

trips.start_date = pd.to_datetime(trips.start_date, format='%m/%d/%Y %H:%M')
trips.end_date = pd.to_datetime(trips.end_date, format='%m/%d/%Y %H:%M')
trips["date"] = trips.start_date.dt.date
trips["year"] = trips.start_date.dt.year
trips["month"] = trips.start_date.dt.month
trips["weekday"]  = trips.start_date.dt.weekday_name
trips["weekday_id"]  = trips.start_date.dt.weekday
trips["hour"] = trips.start_date.dt.hour

#### Ladrones

In [ ]:
# Veamos cuanta gente se 'robó' las bicis
# (Consideramos robada cuando tardó + de 1 semana en devolver)
diasDistintos = trips[trips.start_date.dt.date != trips.end_date.dt.date]
unaSemana = 60 * 24 * 7
ladrones = diasDistintos[diasDistintos.duration > unaSemana]

# No se roban tantas bicis.
ladrones.groupby('year').count()['id']

In [ ]:
#Sólo 13 personas se 'robaron' bicicletas.
ladrones.count()

In [ ]:
ladrones.count()/trips.count()*100

In [ ]:
lg = ladrones.groupby('subscription_type').count()
df_lg = pd.DataFrame({'count' : lg['id']}).reset_index()
df_lg.plot.bar(x='subscription_type',title="Relacion de subscripcion",)

### Normalizamos los datos

In [ ]:
# Nos quedamos con las duraciones < a 6 horas
trips.duration.quantile(0.995)
trips = trips[trips.duration <= 360]

In [ ]:
#Cuantas tipo de subscripciones existen?
trips.groupby('subscription_type').size()
ax = trips.groupby('subscription_type').size().plot.bar(title="Relación de subscripción")
ax.set(ylabel='Cantidad', xlabel='Subscripciones')
ax.set_xticklabels(['Customer', 'Subscriber'], rotation=0)
#Existen 2 tipos de subscripciones: Customer y Subscriber

In [ ]:
#Hacemos otro plot, esta vez de torta, para los tipos de subscripciones.
ax = trips.groupby('subscription_type')\
    .size().plot(kind='pie',title="Relación de subscripción",figsize=(10,10),autopct='%.2f')

In [ ]:
# Las personas sin subscripciones hacen viajes mas largos.

trip_duration_subs = trips.loc[trips.subscription_type == 'Subscriber'].duration.mean()
trip_duration_nonsub = trips.loc[trips.subscription_type == 'Customer'].duration.mean()

print('Duracion de viajes promedio de personas con subscripcion: ' + str(trip_duration_subs))
print('Duracion de viajes promedio de personas sin subscripcion: ' + str(trip_duration_nonsub))

In [ ]:
# Duraciones de los viajes, según Customer o Subscriber
trips['duration_bins'] = pd.cut(trips['duration'], bins=[0, 5, 10, 20, 30, 40, 50, 60, 360])
durationByType = trips.groupby(['subscription_type', 'duration_bins'])['duration'].agg(['count']).reset_index()

ax = sns.factorplot(
    x='duration_bins', y='count', hue='subscription_type', data=durationByType, kind='bar', size=5, aspect=2
)

ax.set_xticklabels(['0-5 min', '5-10 min', '10-20 min', '20-30 min', '30-40 min', '40-50 min', '50-60 min', '60+ min'])
ax.set(ylabel='Cantidad de viajes', xlabel='Duración')
plt.title('Duración de los viajes por tipo de subscripción')

In [ ]:
# Separamos los viajes por tipo de subscripción, se puede ver que los 'Subscriber' lo usan para ir a trabajar
# Mientras que los 'Customer' parecen tener una distribución de campana.
tripsBySubscription = trips.groupby(['subscription_type', 'hour'])['id'].count().reset_index().rename(columns={'id':'count'})

ax = sns.factorplot(
    data=tripsBySubscription, x='hour', y='count', kind='bar', hue='subscription_type', size=6, aspect=2
)
ax.set(ylabel='Cantidad de viajes', xlabel='Horas')
plt.xticks([0, 4, 8, 12, 16, 20, 23], ['0 AM', '4 AM', '8 AM', '12 PM', '4 PM', '8 PM', '23 PM'])
plt.title('Cantidad de viajes por hora')
plt.show()

In [ ]:
# Más indicios de que los Subscriber usan las bicis para trabajar, mientras que los Customer parecen ser 'turistas'
# ya que las utilizan más durante los sábados y domingos.
tripsByDay = trips.groupby(
    ['subscription_type', 'weekday', 'weekday_id']
)['id'].count().reset_index().sort_values('weekday_id').rename(columns={'id': 'count'})

ax = sns.factorplot(
    data=tripsByDay, x='weekday', y='count', hue='subscription_type', kind='bar', size=5, aspect=2
)

ax.set(ylabel='Cantidad de viajes', xlabel='Días')
plt.title('Distribución de viajes en la semana')

In [ ]:
# Los meses de verano tienen un poco más de uso que los demás, tanto para los Customer como para los Subscriber
tripsByMonth = trips.groupby(
    ['subscription_type', 'month']
)['id'].count().reset_index().sort_values('month').rename(columns={'id': 'count'})

sns.factorplot(
    data=tripsByMonth, x='month', y='count', hue='subscription_type', kind='bar', size=5, aspect=2
)
ax.set(ylabel='Cantidad', xlabel='Meses')

In [ ]:
# Estaciones mas populares para el retiro de bicicletas
print ("Top 10 estaciones para retirar bicicletas.")
print (trips.groupby('start_station_name').size().sort_values(ascending=False).nlargest(n=10))

print ('====================')
# Estaciones mas populares para la devolucion de bicicletas
print ("Top 10 estaciones para devolver bicicletas." )
print (trips.groupby('end_station_name').size().nlargest(n=10))

### Viajes en los días de vacaciones

In [ ]:
#Miramos el set de datos para las fechas
#Tenemos datos desde Agosto del 2013 hasta Agosto del 2015
trips['start_date'].sort_values(ascending=True).head(3)

In [ ]:
trips['start_date'].sort_values(ascending=True).tail(3)

In [ ]:
#Filtramos para el anio 2014 y 2015
start_date_holiday = pd.to_datetime('2014-01-01')
end_date_holiday = pd.to_datetime('2014-03-01')
start_date_holiday2 = pd.to_datetime('2015-01-01')
end_date_holiday2 = pd.to_datetime('2015-03-01')

trips_holiday0 = trips[(trips.start_date >= start_date_holiday) & (trips.start_date <= end_date_holiday)]
print ("Cantidad de viajes en vacaciones 2014: " + str(trips_holiday0['id'].count()))

trips_holiday1 = trips[(trips.start_date >= start_date_holiday2) & (trips.start_date <= end_date_holiday2)]
print ("Cantidad de viajes en vacaciones 2015 : " + str(trips_holiday1['id'].count()))

total_trips_holiday = trips_holiday0['id'].count() + trips_holiday1['id'].count()
print ("Total viajes en vacaciones: " + str(total_trips_holiday))

print (' ')
#end 2015
end_2015 = pd.to_datetime('2015-12-31')
trips_NOT_holiday1 = trips[(trips.start_date >= end_date_holiday) & (trips.start_date <= start_date_holiday2)]
trips_NOT_holiday2 = trips[(trips.start_date >= end_date_holiday2) & (trips.start_date <= end_2015)]
trips_NOT_holiday0 = trips[(trips.start_date < start_date_holiday)]

print ("Cantidad de viajes fuera de vacaciones : " + str(trips_NOT_holiday1['id'].count()))
print ("Cantidad de viajes fuera de vacaciones : " + str(trips_NOT_holiday2['id'].count()))
print ("Cantidad de viajes fuera de vacaciones : " + str(trips_NOT_holiday0['id'].count()))

total_trips_NOT_holiday = trips_NOT_holiday0['id'].count()+trips_NOT_holiday1['id'].count()+trips_NOT_holiday2['id'].count()
trips_NOT_holiday0['id'].count()+trips_NOT_holiday1['id'].count()+trips_NOT_holiday2['id'].count()

print ("Total viajes en vacaciones: " + str(total_trips_NOT_holiday))
print (' ')

#Test
print ("Total viajes en vacaciones: " + str(trips['start_date'].count()))
print ("Total viajes en vacaciones: " + 
       str(trips_holiday0['id'].count() + trips_holiday1['id'].count() + 
           trips_NOT_holiday0['id'].count()+trips_NOT_holiday1['id'].count()+trips_NOT_holiday2['id'].count())
      )
#OK

df = pd.DataFrame({'Type':['Holiday','NOT_Holiday'],'count':[total_trips_holiday,total_trips_NOT_holiday]})
df.plot(kind='bar',x=df.Type)

In [ ]:
#Según los totales es de esperar que la cantidad de viajes en los días Holiday sea menor que los que no lo son.
#Vamos a analizar los viajes en cantidad según el tiempo en que relevamos:
#Vemos que los primeros registros datan de: 2013-08-29 09:08:00
#4 meses + 9 meses(04-12 del 2014) + 5 meses (2015-08-31 23:26:00)
#Total = 18 meses
# HOLIDAY: 3 + 3 = 6 meses
df = pd.DataFrame({'Type':['Holiday','NOT_Holiday'],'count_per_month':[total_trips_holiday/18,total_trips_NOT_holiday/6]})

df.plot(kind='bar',x=df.Type)
#Se puede concluir a partir de la relación #viajes/tiempo que en vacaciones(Holiday) hay menos viajes.

In [ ]:
alquiladasPorFecha = pd.DataFrame(trips.groupby('date').count()['id'])

alquiladasPorFecha['date'] = alquiladasPorFecha.index
alquiladasPorFecha.reset_index(drop = True, inplace = True)
alquiladasPorFecha.rename(columns={'id': 'alquiladas'}, inplace=True)
alquiladasPorFecha.date = pd.to_datetime(alquiladasPorFecha.date)
alquiladasPorFecha.plot(x='date', y='alquiladas', figsize=(25,10))

## Exploramos Weather

In [ ]:
weather.dtypes

In [ ]:
# Hay un ingreso por zip_code para cada fecha

print ("Los zip_codes son: ")
print (weather.zip_code.unique())

weather.groupby('date').count()['zip_code'].head()

In [ ]:
# Vemos los datos para cada zip_code
for zc in weather.zip_code.unique():
    print ("zip_code: " + str(zc))
    print (weather[weather.zip_code == zc].isnull().sum())
    print ()

In [ ]:
# Vamos a usar 94107 que es el que tiene los datos más limpios
weather = weather[weather.zip_code == 94107]
weather.drop('zip_code', inplace=True, axis=1)

In [ ]:
weather.events.unique()

In [ ]:
weather.loc[weather.events == 'rain', 'events'] = "Rain"
weather.loc[weather.events.isnull(), 'events'] = "Normal"

In [ ]:
weather.date = pd.to_datetime(weather.date, format='%m/%d/%Y')
weather['year'] = weather.date.dt.year
weather['month'] = weather.date.dt.month
weather['day'] = weather.date.dt.day
weather['day_of_week'] = weather.date.dt.dayofweek

In [ ]:
def toCelsius(fahrenheit):
    return (fahrenheit - 32) / 1.8

def laborables(df):
    calendar = USFederalHolidayCalendar()
    holidays = calendar.holidays(start=df.date.min(), end=df.date.max())
    return df[(~df.date.isin(holidays)) \
            & (df.date.dt.weekday != 5) \
            & (df.date.dt.weekday != 6)]

In [ ]:
# Pasamos las temperaturas a farenheit
for column in weather.columns:
    if "_f" in column:
        weather[column] = weather[column].apply(toCelsius)

weather.rename(columns=lambda x: x.replace("_f", "_c"), inplace=True)

In [ ]:
# Tienen sentido las temperaturas
weather.describe()

In [ ]:
# Relación entre alquiladas y las lluvias
# Se puede ver que en los días de lluvia disminuye el uso de las bicicletas
import matplotlib.patches as mpatches

rainyDays = weather[weather.events.str.contains('Rain')]
rainyDays = laborables(rainyDays)
diasLaborables = laborables(alquiladasPorFecha)
ax = diasLaborables.plot(figsize=(22,12), x='date', y='alquiladas')
for date in rainyDays.date:
    plt.axvspan(date, date, color='red')

plt.title('Relación entre lluvias y cantidad de alquileres')
#ax.add_legend(legend_data='a',title='Lluvias')
red_patch = mpatches.Patch(color='#f76767', label='Lluvias')
blue_patch = mpatches.Patch(color='#6d74ff', label='Alquiladas')
plt.legend(handles=[red_patch, blue_patch])
ax.set(ylabel='Cantidad de alquileres', xlabel='Fecha')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(22,12))

alqLaborables = laborables(alquiladasPorFecha)
wLaborables = laborables(weather)

alqLaborables = alqLaborables[alqLaborables.date.dt.year == 2014]
wLaborables = wLaborables[wLaborables.date.dt.year == 2014]

ax2 = ax1.twinx()
ax1.plot(alqLaborables.date, alqLaborables.alquiladas, label='Alquiladas')
ax2.plot(wLaborables.date, wLaborables.mean_temperature_c, 'g-', label='Temperatura promedio')

ax1.set_xlabel('Fecha')
ax1.set_ylabel('Cantidad alquiladas')
ax2.set_ylabel('Temperatura promedio')
ax1.legend(loc='upper left', shadow=True)
ax2.legend(loc='upper right', shadow=True)
plt.title('Relación entre temperatura promedio y cantidad de alquileres')
plt.show()

#alquiladasPorFecha.plot(x='date', y='alquiladas')
#weather.plot(x='date', y='mean_temperature_c', legend=True, secondary_y=True)

In [ ]:
# Nos quedamos con los días laborables
trips['date'] = pd.to_datetime(trips['date'])
diasLaborables = laborables(trips)
byHour = diasLaborables.groupby(['date', 'hour'])['id'].agg(['count']).sort_index().reset_index()
# Analizamos la temperatura con la que se alquilaron bicicletas durante el día
wl = byHour.merge(weather, on='date')
wl = wl[['date', 'hour', 'count', 'mean_temperature_c']]
wl['temp_entre'] = pd.cut(wl.mean_temperature_c, [0, 12, 24, 30])

In [ ]:
fig = plt.subplots(figsize=(20,10))
ax = sns.stripplot('hour', 'count', data=wl, hue='temp_entre', jitter=True, size=10)
ax.set(ylabel='Bicicletas alquiladas', xlabel='Horas')
ax.set_xticks([0, 4, 8, 12, 16, 20, 23])
ax.set_xticklabels(['0 AM', '4 AM', '8 AM', '12 PM', '4 PM', '8 PM', '23 PM'])
ax.legend(loc='upper right', title='Temperaturas')
plt.title('Bicicletas alquiladas por hora según la temperatura')

## Exploramos Stations

In [ ]:
stations.dtypes
stations.head()

In [ ]:
stations.installation_date = pd.to_datetime(stations.installation_date, format='%m/%d/%Y')

In [ ]:
# Obtenemos la cantidad de viajes entre cada estación
tripsByStation = trips.groupby(['start_station_id', 'end_station_id'])['date'].count().sort_values(ascending=False)
tripsByStation = tripsByStation.to_frame()
tripsByStation.reset_index(inplace=True)
tripsByStation.rename(columns={'date': 'trips'}, inplace=True)

# Obtenemos la distancia entre cada estación
tripsByStation = tripsByStation.merge(stations[['id', 'lat', 'long']], left_on='start_station_id', right_on=['id'])
tripsByStation.drop('id', axis=1, inplace=True)
tripsByStation.rename(columns={'lat': 'lat_start', 'long': 'long_start'}, inplace=True)
tripsByStation = tripsByStation.merge(stations[['id', 'lat', 'long']], left_on='end_station_id', right_on=['id'])
tripsByStation.drop('id', axis=1, inplace=True)
tripsByStation.rename(columns={'lat': 'lat_end', 'long': 'long_end'}, inplace=True)

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
# Calculamos la distancia entre las estaciones y eliminamos las columnas que no nos sirven más
tripsByStation['distance'] = tripsByStation.apply(lambda row: haversine(row.long_start, row.lat_start, row.long_end, row.lat_end), axis=1)
tripsByStation.drop(['lat_start', 'lat_end', 'long_start', 'long_end'], inplace=True, axis=1)

In [ ]:
# Los 5 viajes más comunes
tripsByStation.sort_values(by='trips', ascending=False, inplace=True)
tripsByStation.head()

In [ ]:
# Agregamos la distancia de cada viaje en trips.
trips = trips.merge(
    tripsByStation[['start_station_id', 'end_station_id', 'distance']], 
    on=['start_station_id', 'end_station_id']
    )

In [ ]:
# Agregamos la velocidad de cada viaje
trips['velocity'] = trips.apply(lambda row: round(row.distance * 1000 / (row.duration * 60), 2), axis=1)

In [ ]:
# Veamos los valores de velocidad
trips.sort_values(by='velocity', ascending=False)

In [ ]:
# Vamos a quedarnos con valores 'normales': 
# La velocidad (Un ciclista profesional llega a 11 m/s): 
# eliminando los mayores a 8 m/s sacamos menos del 0.5% de los datos.
# Distancias menores a 6km
trips.velocity.quantile(0.995)
trips = trips[trips.velocity < 8]

trips.distance.quantile(0.995)
trips = trips[trips.distance < 6]

In [ ]:
# Cuáles fueron las bicicletas más usadas (en cantidad y en distancia)
mostUsed = trips.groupby('bike_id')['distance']\
    .agg(['sum', 'count', 'mean'])\
    .reset_index()

mostUsed.sort_values(by='sum', ascending=False).head(10)
mostUsed.sort_values(by='count', ascending=False).head(10)

In [ ]:
# Calculamos las distancias recorridas cada día
distanceByDate = trips.groupby('date')['distance'].agg(['sum', 'count', 'mean']).reset_index()
distanceByDate.date = pd.to_datetime(distanceByDate.date)

# Promedio de distancias recorridas en los días de semana del 2014
distanceByDate[(distanceByDate.date.dt.year == 2014)   \
               & (distanceByDate.date.dt.weekday != 5) \
               & (distanceByDate.date.dt.weekday != 6)]\
    .plot(x='date', y='mean', figsize=(20,10))

In [ ]:
# Vamos a ver la velocidad promedio de los Customer vs Subscriber
trips.groupby('subscription_type')['velocity'].mean()

In [ ]:
fig = plt.subplots(figsize=(20,10))
ax = sns.stripplot('weekday_id', 'velocity', data=trips, hue='subscription_type', jitter=True)
ax.set_xticklabels(['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'])
ax.set(ylabel='Velocidad (m/s)', xlabel='')
ax.legend(loc='upper left')
plt.title('Velocidad por día de la semana')